In [ ]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
%matplotlib inline

In [ ]:
train_path = 'drive/My Drive/shoes/train'
valid_path = 'drive/My Drive/shoes/valid'
test_path = 'drive/My Drive/shoes/test'

In [ ]:
if K.image_data_format()== 'channels_first':
  input_shape = (3,150,150)
else:
  input_shape = (150,150,3)

In [ ]:
train_data_generator = ImageDataGenerator(rescale = 1. /225, shear_range = 0.2, zoom_range = 0.2, horizontal_flip=True)
valid_data_generator = ImageDataGenerator(rescale = 1. /255)

train_data = train_data_generator.flow_from_directory(train_path, target_size = (150,150), classes = ['adidas','nike','puma'], batch_size = 10)
valid_data = valid_data_generator.flow_from_directory(valid_path, target_size = (150,150), classes = ['adidas','nike','puma'], batch_size = 5)
test_data = ImageDataGenerator().flow_from_directory(test_path, target_size = (150,150), classes = ['adidas','nike','puma'], batch_size = 5)


In [ ]:
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
imgs, labels = next(train_data)
plots(imgs, titles = labels)

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape= input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(3))
model.add(Activation('softmax'))

model.summary()


In [ ]:
model.compile(Adam(lr= 0.0001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_data, steps_per_epoch= 87, validation_data = valid_data, validation_steps = 21, epochs = 50, verbose = 2)

In [ ]:
model.save_weights('fromscratch_first_run.h5')

In [ ]:
img_pred = image.load_img('tt4.jpg', target_size = (150,150))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred ,axis = 0)

result = model.predict(img_pred)
# print(result)


if result[0][0] == 1:
  ans = 'Adidas'
elif result[0][1] == 1:
  ans = 'Nike'
else:
  ans = 'Puma'
print(ans)


In [ ]:
test_imgs, test_labels = next(test_data)
plots(test_imgs, titles = test_labels)

In [ ]:
predictions = model.predict_generator(test_data, steps = 1, verbose = 0)
predictions

In [ ]:
#y_true = ['adidas', 'nike', 'puma', 'adidas', 'adidas', 'puma']
#y_pred = ['nike', 'nike', 'puma', 'nike', 'nike', 'puma']

#classes=['adidas', 'nike', 'puma']

#confusion_matrix(y_true, y_pred, labels=['adidas', 'nike', 'puma'])

In [ ]:
'''def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()'''

In [ ]:
'''cnf_matrix = confusion_matrix(valid_data, test_data,labels=['adidas', 'nike', 'puma'])
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['adidas', 'nike', 'puma'],
                      title='Confusion matrix, without normalization')'''

In [ ]:
#cm_plot_labels = ['adidas','nike','puma']
#plot_confusion_matrix(cm, cm_plot_labels, title = 'Confusion Matrix')